#### Reading the data we prepared:

In [87]:
import pandas as pd, numpy as np
np.random.seed(42)

data_source = '../cleaned_Airline_Delay_Data.csv'
df = pd.read_csv(data_source)
X = df.drop(columns=['arr_del15'])
y = df['arr_del15']

print(f"Data shape: {X.shape}")
print(f"Features: {len(X.columns)}")
print(f"\nCategorical columns for catboost: {X.select_dtypes(include='object').columns.tolist()}")
print(f"Numeric columns: {len(X.select_dtypes(exclude='object').columns)}")

Data shape: (171223, 35)
Features: 35

Categorical columns for catboost: ['carrier_name', 'airport_name']
Numeric columns: 33


In [93]:
cat_boost_cols = ['year', 'carrier_name', 'airport_name', 'arr_flights', 'isCovid', 'isPeakSeason', 'isLowSeason',
                  'is_top5', 'is_holiday', 'month_sin', 'month_cos', 'is_GA_Atlanta']

In [ ]:
# I will split X and y into training and test sets according to years as this is time-series data. 
# this datasets spans 10 years so we train with 7, test with 3 years of data.
train_mask = X['year'] <= 2020
X_train, y_train = X[train_mask], y[train_mask]

test_mask = X['year'] > 2020
X_test, y_test = X[test_mask], y[test_mask]

#### In the previous notebook, I kept many features, but it’s often **better to select the most predictive ones rather than using all features blindly, as this reduces noise and improves efficiency**.
Firstly, remove collinearity

In [90]:
from feature_engine.selection import DropCorrelatedFeatures

num_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
num_cols = [c for c in num_cols if c not in ['year']]

print(f"Checking correlation for {len(num_cols)} numeric features...")

tr = DropCorrelatedFeatures(variables=num_cols, method='pearson', threshold=0.8)
Xt = tr.fit_transform(X_train[num_cols])

if len(tr.correlated_feature_sets_) > 0:
    print(f"\nFound {len(tr.correlated_feature_sets_)} correlated feature sets:")
    for feature_set in tr.correlated_feature_sets_:
        print(f"  - {feature_set}")
else:
    print("No highly correlated features (threshold=0.8)")

Checking correlation for 32 numeric features...
No highly correlated features (threshold=0.8)


#### --> It seems none of our features are too **linearly** correlated.

#### From our EDA, we created two airport encoding strategies:
- **Atlanta only**: Binary feature for highest-importance airport
- **Top 5**: Binary feature for top 5 airports by importance

Let's test which performs better using baseline Random Forest.

In [98]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

cat_cols = ['carrier_name', 'airport_name']

X_train_atlanta = X_train.drop(columns=cat_cols + ['is_top5'])
X_testatlanta = X_test.drop(columns=cat_cols + ['is_top5'])

X_train_top5 = X_train.drop(columns=cat_cols + ['is_GA_Atlanta'])
X_testtop5 = X_test.drop(columns=cat_cols + ['is_GA_Atlanta'])

X_train_both = X_train.drop(columns=cat_cols)
X_testboth = X_test.drop(columns=cat_cols)

results = {}

for name, X_tr, X_te in [
    ('Atlanta_only', X_train_atlanta, X_testatlanta),
    ('Top5_only', X_train_top5, X_testtop5),
    ('Both', X_train_both, X_testboth)
]:
    print(f"Training with {name}... ({X_tr.shape[1]} features)", end=' ')
    
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=12,
        min_samples_split=10,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_tr, y_train)
    
    # Predictions
    train_pred = model.predict(X_tr)
    test_pred = model.predict(X_te)
    
    results[name] = {
        'Train_R2': r2_score(y_train, train_pred),
        'Test_R2': r2_score(y_test, test_pred),
        'MAE': mean_absolute_error(y_test, test_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, test_pred))
    }
    print(f"✓ Test R² = {results[name]['Test_R2']:.4f}")

results_df = pd.DataFrame(results).T
print("AIRPORT ENCODING COMPARISON - Random Forest Baseline")
print("="*70)
print(results_df.round(4))

best_strategy = results_df['Test_R2'].idxmax()
print(f"\n✓ Best strategy: {best_strategy} (R² = {results_df.loc[best_strategy, 'Test_R2']:.4f})")


if best_strategy == 'Atlanta_only':
    X_train = X_train.drop(columns=['carrier_name', 'airport_name', 'is_top5'], inplace=True)
    X_test = X_test.drop(columns=['carrier_name', 'airport_name', 'is_top5'], inplace=True)

elif best_strategy == 'Top5_only':
    X_train = X_train.drop(columns=['carrier_name', 'airport_name', 'is_GA_Atlanta'], inplace=True)
    X_test = X_test.drop(columns=['carrier_name', 'airport_name', 'is_GA_Atlanta'], inplace=True)

else:  
    X_train = X_train.drop(columns=['carrier_name', 'airport_name'], inplace=True)
    X_test = X_test.drop(columns=['carrier_name', 'airport_name'], inplace=True)

Training with Atlanta_only... (32 features) ✓ Test R² = 0.8089
Training with Top5_only... (32 features) ✓ Test R² = 0.7971
Training with Both... (33 features) ✓ Test R² = 0.8012
AIRPORT ENCODING COMPARISON - Random Forest Baseline
              Train_R2  Test_R2      MAE     RMSE
Atlanta_only    0.9500   0.8089  25.3420  78.6362
Top5_only       0.9525   0.7971  25.7288  81.0308
Both            0.9526   0.8012  25.5581  80.2153

✓ Best strategy: Atlanta_only (R² = 0.8089)


**Most Predictive:** Atlanta Only

This validates our statistical analysis from the EDA notebook. The importance scores correctly identified which airports contribute most to predictive performance.


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold

# Prepare datasets based on best strategy
if best_strategy == 'Atlanta_only':
    X_train_opt = X_train.drop(columns=['carrier_name', 'airport_name', 'is_top5'])
    X_test_opt = X_test.drop(columns=['carrier_name', 'airport_name', 'is_top5'])
elif best_strategy == 'Top5_only':
    X_train_opt = X_train.drop(columns=['carrier_name', 'airport_name', 'is_GA_Atlanta'])
    X_test_opt = X_test.drop(columns=['carrier_name', 'airport_name', 'is_GA_Atlanta'])
else:  # Both
    X_train_opt = X_train.drop(columns=['carrier_name', 'airport_name'])
    X_test_opt = X_test.drop(columns=['carrier_name', 'airport_name'])

print(f"Optimizing with {best_strategy} strategy ({X_train_opt.shape[1]} features)")

# Random Forest objective
def rf_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 8, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 10, 50),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 4, 20),
        'max_features': trial.suggest_float('max_features', 0.3, 0.9),
        'random_state': 42,
        'n_jobs': -1
    }
    
    model = RandomForestRegressor(**params)
    
    # 3-fold CV on training set
    scores = cross_val_score(
        model, X_train_opt, y_train,
        cv=3, scoring='r2', n_jobs=-1
    )
    
    return scores.mean()


X_train_cb = X_train[cat_boost_cols]
X_test_cb = X_test[cat_boost_cols]

for col in cat_boost_cols:
    X_train_cb[col] = X_train_cb[col].astype(str)

cat_features = ['carrier_name', 'airport_name']

def catboost_objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "loss_function": "RMSE",
        "verbose": 0
    }

    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X_train_cb):
        X_tr, X_val = X_train_cb.iloc[train_idx], X_train_cb.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model = CatBoostRegressor(**params, cat_features=cat_boost_cols)
        model.fit(X_tr, y_tr)

        preds = model.predict(X_val)
        scores.append(r2_score(y_val, preds))

    return np.mean(scores)

print("\n" + "="*70)
print("HYPERPARAMETER OPTIMIZATION")
print("="*70)

print("\n1. Optimizing Random Forest...")
rf_study = optuna.create_study(direction='maximize', study_name='RandomForest')
rf_study.optimize(rf_objective, n_trials=30, show_progress_bar=True)

print(f"\n✓ Best RF CV R²: {rf_study.best_value:.4f}")
print(f"Best params: {rf_study.best_params}")

print("\n2. Optimizing CatBoost...")
cb_study = optuna.create_study(direction="maximize")
cb_study.optimize(catboost_objective, n_trials=30)

print(f"\n✓ Best CatBoost CV R²: {cb_study.best_value:.4f}")
print(f"Best params: {cb_study.best_params}")

Optimizing with Atlanta_only strategy (32 features)


C:\Users\User\AppData\Local\Temp\ipykernel_10468\1518932335.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_cb[col] = X_train_cb[col].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_10468\1518932335.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_cb[col] = X_train_cb[col].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_10468\1518932335.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde


HYPERPARAMETER OPTIMIZATION

2. Optimizing CatBoost...


In [122]:
X_train_cb.columns

Index(['year', 'carrier_name', 'airport_name', 'arr_flights', 'isCovid',
       'isPeakSeason', 'isLowSeason', 'is_holiday', 'month_sin', 'month_cos',
       'is_GA_Atlanta'],
      dtype='object')